# Gaussian Fit Demonstration

This notebook shows how to perform spectral deconvolution using the FTIR tool suite developed by KBI Biopharma. 

The spectral deconvolution tools utilize the scipy solvers to deconvolute second derivative or fourier self deconvolution spectra using a guassian peak model. Gradiant based solvers can be utilized using the `gaussian_miminize`. The parameter space is very large, and thus different gradient based solvers are likely to converge to slightly different local minima. There are three parameters per peak (mean, height and width), and thus 36 inputs are present for a 12 peak reference set. The default peak definitions `yang_h20_2015` has 14 peaks, and thus 42 input parameters are present. A stochastic method, `gaussian_differential_evolution`, can be used to try to attempt to find the global minimum within the defined parameters space, however this may take a very long time, and may not completely converge.

In [1]:
# imports
%matplotlib inline
import ipywidgets as widgets
import plotly.graph_objects as go
import peakutils

%matplotlib inline
import matplotlib.pyplot as plt

from IPython.display import clear_output
from IPython.display import display, HTML

import pandas as pd
import numpy as np
import tempfile
import os
from ftir.modeling.buffer_subtraction import find_buffer_subtraction_constant, buffer_subtract
from ftir.modeling.peak_fitting import gaussian_minimize, gaussian_differential_evolution, gaussian_least_squares
from ftir.modeling.peak_fitting import secondary_structure, create_fit_plots, gaussian_list
from ftir.modeling.peak_definitions import yang_h20_2015
from ftir.modeling.area_norm import area_norm
from ftir.modeling.derivative import find_deriv, sd_baseline_correction
from ftir.io.utils import create_df_from_single_file
from ftir.io.file_importer import file_select, file_upload, graphical_VM_file_importer
from ftir.io.select_traces import multi_checkbox_grapher
from norbi.ui.plotly_plots import baseline_check_graph, raw_data_graph_inset4, peak_ctr_check_graph3b, raw_data_graph6

from norbi.normalize.df_norm import df_trunc, df_center, df_area_norm

### Import from a local file (zipped data folder or MS Excel file) filesize limit ~15MB

In [2]:
def SelectMultipleInteract(df_y_labels):
    
    global radio_selections

    radio_selections = []
    plot_output.clear_output()
    with plot_output:
        
        radio_sel_order = widgets.RadioButtons(
                    options=df.columns.to_list()[1:],
                #     value='pineapple',
                    description='Click each to set plot order:',
                    disabled=False,
                    style = {'description_width': 'initial'},
                    layout=widgets.Layout(width='750px', margin='0 0 5px 0')
                )

        offset_interval=widgets.FloatSlider(value=0, min=0, max = 100, step = 0.1, description='Offset',
                                                   layout =widgets.Layout(width='400px', margin='0 0 0 0'), continuous_update=False)
        reset_btn = widgets.Button(
                            description='Click to Reselect Plots',
                            disabled=False,
                            layout=widgets.Layout(width='200px', margin='0 0 20px 0'), #top, right
                            button_style='success', # 'success', 'info', 'warning', 'danger' or ''
                            tooltip='Click me',
                            icon=''
                        )
        pk_max_norm = widgets.Checkbox(value=False,
                                    description='Normalize all plots to peak max',
                                    disabled=False,
                                    indent=True,
                                    style = {'description_width': 'initial'},
                                    layout=widgets.Layout(width='250px', margin='0 0 0 0')
                                )        

        cut = widgets.FloatRangeSlider(
                                    value=[df.iloc[0,0], df.iloc[-1,0]],
                                    min=df.iloc[0,0],
                                    max=df.iloc[-1,0],
                                    step=0.1,
                                    description='Truncation limits:',
                                    style = {'description_width': 'initial'},
                                    disabled=False,
                                    continuous_update=False,
                                    orientation='horizontal',
                                    readout=True,
                                    readout_format='.1f',
                                    layout=widgets.Layout(width='800px', margin='0 0 5px 0')
                                )
        zoom_level=widgets.FloatSlider(value=1, min=1, max = 100.0, step=1, description='Zoom',
                                           layout =widgets.Layout(width='400px'), continuous_update=False) 
        
        linewidth=widgets.FloatSlider(value=1.2, min=0.1, max = 5.0, step=0.1, description='Line Width',
                                           layout =widgets.Layout(width='300px'), continuous_update=False) 
        area_norm = widgets.Checkbox(value=False,
                                    description='Area Normalize?',
                                    style = {'description_width': 'initial'},
                                    disabled=False,
                                    indent=True,
                                    layout=widgets.Layout(width='250px', margin='0 0 0 0'))        
        deriv = widgets.Checkbox(value=False,
                                    description='2nd Derivative?',
                                    style = {'description_width': 'initial'},
                                    disabled=False,
                                    indent=True,
                                    layout=widgets.Layout(width='250px', margin='0 0 0 0'))  
        baseline = widgets.Checkbox(value=False,
                                    description='Baseline?',
                                    style = {'description_width': 'initial'},
                                    disabled=False,
                                    indent=True,
                                    layout=widgets.Layout(width='250px', margin='0 0 0 0'))  
        window = widgets.Text(
                        value='21',
                        placeholder='Type something',
                        description='Deriv window length:',
                        disabled=False   
                    )
        global y_label, x_label
        y_label = widgets.Text(
                        value='Absorbance',
                        placeholder='Type something',
                        description='y-axis label:',
                        disabled=False   
                    )
        x_label = widgets.Text(
                        value='Retention Time (min)',
                        placeholder='Type something',
                        description='x-axis label:',
                        disabled=False   
                    )
        ui5 = widgets.HBox([y_label, x_label])        
        ui1 = widgets.HBox([radio_sel_order, ])
        ui2 = widgets.HBox([cut, linewidth])
        ui3 = widgets.HBox([zoom_level, offset_interval, window])
        ui4 = widgets.HBox([reset_btn, pk_max_norm, area_norm, deriv, baseline])

        ui = widgets.VBox([ui1, ui4, ui2, ui3])
        def select_and_truncate(radio_sel_order, offset_interval, cut, pk_max_norm, linewidth, zoom_level, y_label, x_label, area_norm, baseline, deriv, window):
            radio_selections.append(radio_sel_order)
            def on_reset_btn_clicked(b):
                radio_selections.clear()
                out.clear_output()
            reset_btn.on_click(on_reset_btn_clicked)
            cut1, cut2 = cut
            sel_samps = radio_selections.copy()
            sel_samps = list(dict.fromkeys(sel_samps))
            sel_samps.insert(0, df.columns[0])
            global df_truncated
            if pk_max_norm:
                # normalize uv and tic peaks to same arbitrary scale
                df_sel = df.copy()
                for i in range(len(df_sel.columns)-1):
                    max_y = df_sel.iloc[:,i+1].max() 
                    df_sel.iloc[:,i+1] = df_sel.iloc[:,i+1]/max_y

            else:
                df_sel = df.copy()
            df_truncated = df_trunc(df_sel[sel_samps], cut1, cut2,)
            df_truncated.reset_index(drop=True, inplace=True)

                
            if deriv:
                df_truncated = find_deriv(df_truncated, flip=True, window_length=int(window))
            if baseline:
                df_truncated = sd_baseline_correction(df_truncated, flip=False, 
                           method='rubberband', bounds=[df_truncated.iloc[0,0], df_truncated.iloc[-1,0]], inplace=False)
            if area_norm:
                df_truncated = df_area_norm(df_truncated.copy())   
            raw_data_graph6(df_truncated[sel_samps],offset_interval,  linewidth=linewidth, zoom_level=zoom_level, y_label=y_label, x_label=x_label)
        out = widgets.interactive_output(select_and_truncate,{
                                         'radio_sel_order':radio_sel_order,
                                         'offset_interval':offset_interval,
                                         'cut':cut,
                                         'pk_max_norm':pk_max_norm,
                                         'linewidth':linewidth,
                                         'zoom_level':zoom_level,
                                         'y_label':y_label,
                                         'x_label':x_label,
                                         'area_norm':area_norm,
                                         'baseline':baseline,
                                         'deriv':deriv,
                                         'window':window}                                          
                                        )
        display(ui, out, ui5)

In [3]:
# Upload csv file containing data for analysis
sel_file = widgets.FileUpload(accept= '.csv')

# button to create dataframe
create_df_button = widgets.Button(description="Create plots",
                                  layout=widgets.Layout(width='200px', margin='2px 0 0 20px')) #top, 
# create plot output widget to enable display of graphs
plot_output = widgets.Output()

# function to create dataframe from uploaded temp file when button clicked
def on_create_df_button_clicked(b):
    import tempfile
    global df
    f = sel_file.data[0]
    TEMPDIR=tempfile.TemporaryFile()
    TEMPDIR.write(f)
    TEMPDIR.seek(0)
    ef = pd.read_csv(TEMPDIR)
    TEMPDIR.close()
    df = pd.DataFrame(ef) 
    df_y_labels = df.columns.to_list()[1:]
    SelectMultipleInteract(df_y_labels)

# call dataframe creation with button click
create_df_button.on_click(on_create_df_button_clicked)

select_cgms = widgets.HBox([sel_file, create_df_button])
widgets.VBox([select_cgms, plot_output])

In [69]:
import pandas as pd
import numpy as np

from plotly.graph_objs import Layout
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pltcolors = ['#1f77b4', '#ff7f0e','#2ca02c','#d62728','#9467bd','#8c564b','#e377c2','#7f7f7f','#bcbd22','#17becf',
       '#1f77b4', '#ff7f0e','#2ca02c','#d62728','#9467bd','#8c564b','#e377c2','#7f7f7f','#bcbd22','#17becf',
       '#1f77b4', '#ff7f0e','#2ca02c','#d62728','#9467bd','#8c564b','#e377c2','#7f7f7f','#bcbd22','#17becf',
       '#1f77b4', '#ff7f0e','#2ca02c','#d62728','#9467bd','#8c564b','#e377c2','#7f7f7f','#bcbd22','#17becf',
       '#1f77b4', '#ff7f0e','#2ca02c','#d62728','#9467bd','#8c564b','#e377c2','#7f7f7f','#bcbd22','#17becf',
       '#1f77b4', '#ff7f0e','#2ca02c','#d62728','#9467bd','#8c564b','#e377c2','#7f7f7f','#bcbd22','#17becf',
              ]

from norbi.normalize.df_norm import df_trunc
# Method to define plotly.graph_objects plotting format


class plotly_go:
    def __init__(
        self, left=False, right=False, min_y=0, max_y=100
    ):  # Runs every time we run plotly_go

        self.left = left  # Self refers to the instance, i.e. emp_1.first
        self.right = right
        self.min_y = min_y
        self.max_y = max_y
        self.layout = Layout(
            width = 1200,
            height = 500,
            margin=dict(l=120, r=20, t=20, b=20),
            paper_bgcolor="white",
            plot_bgcolor="white",
            xaxis={
                "showgrid": False,
                "showline": True,
                "linewidth": 2,
                "linecolor": "black",
                "mirror": True,
                "title_text": "",
                "range": [self.left, self.right],
                "ticks": "outside",
                "tickcolor": "black",
                "tickwidth": 2,
                "ticklen": 10,

            },
            yaxis={
                "showgrid": False,
                "showline": True,
                "linewidth": 2,
                "linecolor": "black",
                "mirror": True,
                "title_text": "Abs",
                # "range": [self.min_y, self.max_y],
                "ticks": "outside",
                "tickcolor": "black",
                "tickwidth": 2,
                "ticklen": 10,
            },
        )

class Gaussian_plots(plotly_go):
    def __init__(
        self, df, col, peak_list, linewidth=1.2, 
    ):
        super().__init__(
          
        )  
        # Allows Employee class to pass info
        
        # Call graph object figure initialization
        fig = make_subplots(rows=2) 

        
        xdata = df.iloc[:,0]
        y_fit = sum(peak_list)
        # Add traces
        fig.add_trace(go.Scatter(x = xdata, y=df[col],
                            mode='lines',
                            name='original'), row=1, col=1)
        fig.add_trace(go.Scatter(x = xdata, y=y_fit,
                            mode='lines',
                            name='Model fit'), row=1, col=1)
        for i in range(len(peak_list)):
            fig.add_trace(go.Scatter(x=xdata, y=peak_list[i],  showlegend= False, line=dict( width=linewidth, dash='dash') ), row=1, col=1)
        
        # Add subplot with residuals
        resid = df[col] - y_fit
        fig.add_trace(go.Scatter(x = xdata, y=resid,
                            mode='lines',
                            name='residuals'),  row=2, col=1)         
        fig['layout'].update(self.layout)
        fig.update_layout(xaxis=dict(tickmode="auto", tickangle=0))  
        fig.layout.yaxis2.update({"showgrid": False,
                "showline": True,
                "linewidth": 2,
                "linecolor": "black",
                "mirror": True,
                "title_text": "",
                "ticks": "outside",
                "tickcolor": "black",
                "tickwidth": 2,
                "ticklen": 10,})
        fig.layout.xaxis2.update({"showgrid": False,
                "showline": True,
                "linewidth": 2,
                "linecolor": "black",
                "mirror": True,
                "title_text": "",
                "ticks": "outside",
                "tickcolor": "black",
                "tickwidth": 2,
                "ticklen": 10,})
        pio.show(fig)    



## Fitting using least squares solver
Suggested method based on literature.  Assumes area is normalized to 1.0

Why do we use a least square solver?

In [75]:
def CurveFitInteract():
    x_val = np.asarray(df_truncated.iloc[:,0])
    df_truncated_y_labels = df_truncated.columns.to_list()[1:]
    initial_selected_trace = df_truncated.columns.to_list()[1]
    curve_fit_output.clear_output()
    with curve_fit_output:  
        selected_trace=widgets.RadioButtons(
                                            options=df_truncated_y_labels,
                                            
                                            description='Select traces',
                                            disabled=False,
                                            layout=widgets.Layout(width='350px', margin='0 0 5px 0')
                                            )

        peak_width=widgets.FloatSlider(value=5, min=1, max = 20.0, step=0.1, description='Peak Width',
                                           layout =widgets.Layout(width='400px'), continuous_update=False) 

        
        linewidth=widgets.FloatSlider(value=1.2, min=0.1, max = 5.0, step=0.1, description='Line Width',
                                           layout=widgets.Layout(width='300px'), continuous_update=False) 
               
        ui1 = widgets.HBox([selected_trace, peak_width])
                                 
        def baseline_single_plot(selected_trace, peak_width, linewidth):
            global df_curve_fit, gaussian_list_data                    
            #create shortened baseline for interactive plot to avoid early negative baseline dips from solute flowthrough
            df_curve_fit = df_truncated.copy()
            area, res = gaussian_least_squares(df_curve_fit, selected_trace, peaks=yang_h20_2015, peak_width=peak_width, params={'loss':'linear'})
            gaussian_list_data = gaussian_list(df_curve_fit.iloc[:,0], *res.x)
            plot_output.clear_output()
            with plot_output:
#                 plt = create_fit_plots(df_curve_fit, selected_trace, gaussian_list_data)
                Gaussian_plots(df_curve_fit, selected_trace, gaussian_list_data,  linewidth=linewidth)

    #             plt.savefig('/home/jovyan/ftir_data_analytics/data/fitted_results/'+sample+'.png', dpi=600)
                plt.show()
            df_output.clear_output()
            with df_output:
                structs = secondary_structure(area, yang_h20_2015)
                global structs_df
                structs_df = pd.DataFrame(list(structs.items()), columns=['Structure','Fraction'])
                display(HTML(structs_df.to_html()))               
        
        out_base = widgets.interactive_output(baseline_single_plot, { 
                                         'selected_trace':selected_trace,
                                         'peak_width':peak_width,
                                         'linewidth':linewidth})

        display(ui1, plot_output, out_base, df_output)

In [71]:
curve_fit_button = widgets.Button(description="Curve fit plots",
                                  layout=widgets.Layout(width='200px', margin='10px 10px 0 0'))
curve_fit_output = widgets.Output()
plot_output= widgets.Output()
df_output= widgets.Output()
# function to create dataframe from uploaded temp file when button clicked
def on_curve_fit_button_clicked(b):
    CurveFitInteract()

# call dataframe creation with button click
curve_fit_button.on_click(on_curve_fit_button_clicked)

pointers_text = '''WARNING:  Fits may not always converge, which may crash your web browser, requiring restart <br>
                    (if this happens, try narrowing the peak fitting region)'''

curve_fit_pointers = widgets.HTML(
    value=pointers_text,
    placeholder='',
    description='',
)

initiate_curve_fit = widgets.HBox([curve_fit_button, curve_fit_pointers])
widgets.VBox([initiate_curve_fit, curve_fit_output])

In [72]:

import base64
import pandas as pd
from IPython.display import HTML
# html_out = widgets.Output()
def create_download_link( df, title, filename):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)


In [78]:
download_button = widgets.Button(description="Download",
                                  layout=widgets.Layout(width='200px', margin='0 0 0 0'))
download_output = widgets.Output()
# function to create dataframe from uploaded temp file when button clicked
def on_download_button_clicked(b):
    download_output.clear_output()
    with download_output:
        display(create_download_link(structs_df, title = "Click here to initiate structure_summary.csv download", filename = "structure_summary.csv"))        
#         display(create_download_link(gaussian_list_data, title = "Click here to initiate curve_fit_plots.csv download", filename = "curve_fit_plots.csv"))
        display(create_download_link(df_truncated, title = "Click here to initiate truncated_plots.csv download", filename = "truncated_plots.csv"))   

# call dataframe creation with button click
download_button.on_click(on_download_button_clicked)

widgets.VBox([download_button, download_output])